In [1]:
## import library 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import re

import json     
import time 
from datetime import date
import os 
import warnings
warnings.filterwarnings('ignore')

import psycopg2
import os

In [2]:
flatform = 'batdongsan'
today = date.today() 
name = 'huong'
order = 0

path_txt = f'd:/DUE/AI_Real_Estate/data/source/{flatform}/{today}'

try:
    os.mkdir(path_txt)
except:
    pass 

In [3]:
## attribute for selenium
opt = Options()
opt.add_argument("--disable-infobars")
opt.add_argument("start-maximized")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-notifications")
opt.add_argument("--disable-gpu")
# opt.add_argument("--headless")
opt.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 1,       
    "profile.default_content_setting_values.media_stream_camera": 1,
    "profile.default_content_setting_values.geolocation": 1, 
    "profile.default_content_setting_values.notifications": 1 
  })


In [4]:
## define function 
def create_driver(): 
    driver = webdriver.Chrome(options = opt)
    return driver

def count_page(url): 
    driver = create_driver()
    driver.get(url)
    return int(driver.find_elements(By.CLASS_NAME, 're__pagination-number')[-1].text.replace('.', ''))

def collect_item(driver, part):
    lst_url = []
    if part == 'du-an-bat-dong-san': 
        driver.execute_script("window.scrollBy(0, 100);")
        lst_items = driver.find_elements(By.CLASS_NAME, 'js__project-card.js__card-project-web.re__prj-card-full')
        for item in lst_items: 
            lst_url.append(item.find_element(By.TAG_NAME, 'a').get_attribute('href'))
    else:
        lst_items = driver.find_element(By.ID, 'product-lists-web').find_elements(By.CLASS_NAME, 'js__product-link-for-product-id')
        for item in lst_items: 
            lst_url.append(item.get_attribute('href'))
    
    return lst_url

def save_html(page_source, post_id):    
    html = BeautifulSoup(page_source, 'html5lib').find('div', class_ = 're__pr-info pr-info js__product-detail-web').prettify()
    
    name_txt = f'{path_txt}/{post_id}.txt'
    with open(name_txt, "w", encoding = 'utf-8') as f:
        f.write(html)
    return

def crawl_data(driver, news_type): 
    record = {}
    
#     post_id = driver.current_url.split('-')[-1].split('pr')[1]
    post_id = driver.current_url.split('-')[-1]
    post_id = re.sub(r'\D', '', post_id)
    
    record['post_id'] = post_id
    record['Loại tin'] = news_type
    record['Tiêu đề'] = driver.find_element(By.TAG_NAME, 'h1').text.strip() 
    record['Địa chỉ'] = driver.find_element(By.CLASS_NAME, 're__pr-short-description.js__pr-address').text.strip()
    record['Mô tả'] = driver.find_element(By.CLASS_NAME, 're__section-body.re__detail-content.js__section-body.js__pr-description.js__tracking').text.strip()
    
    keys = [i.text for i in driver.find_elements(By.CLASS_NAME, 're__pr-specs-content-item-title')]
    values = [i.text for i in driver.find_elements(By.CLASS_NAME, 're__pr-specs-content-item-value')]
    for key, value in zip(keys, values): 
        record[key.strip()] = value.strip()
    
    record['URL'] = driver.current_url
    record['date'] = str(today)
    
    try:
        tracking_id = "user-listing-ldp" 
        contact = driver.find_element(By.CSS_SELECTOR, f"[tracking-id='{tracking_id}']")
        record['Liên hệ'] = contact.text.strip()
        record['card_visit'] = contact.get_attribute("href")
    except :
        pass
    
    save_html(driver.page_source, post_id)
    
    return record

def process(news_type, part, iter = None):
    lst_data = []
    
    if iter == None: 
        print('No data in crawling process!')
        return
    
    for id_page in range(1,15):  #limit page  
    # for id_page in range(1, iter):    
        driver = create_driver()
        url = f'https://batdongsan.com.vn/{part}-da-nang/p{id_page}'
        print(f'Handling on page {id_page}')
        driver.get(url)
        lst_url = collect_item(driver, part)
        driver.quit()
        
        for url in lst_url: 
            print(f'Crawling on {url}')
            
            bot = create_driver()
            bot.get(url)
            record = crawl_data(bot, news_type)
            lst_data.append(record)
            bot.quit()
    return lst_data

In [5]:
## crawling 
name_file = f'd:/DUE/AI_Real_Estate/data/record/{flatform}/{today}'
exist_data = [] 

try:
    os.mkdir(name_file)
except:
    pass 

exist_data = []     
try:
    with open(f'{name_file}/{name}_{order}.json', "r", encoding="utf-8") as file:
        exist_data = json.load(file)
except FileNotFoundError:
    pass

types_news = {'Nhà đất bán' : 'nha-dat-ban', 'Nhà đất cho thuê' : 'nha-dat-cho-thue'}
# types_news = {'Nhà đất bán' : 'nha-dat-ban'}

try: 
    for type, part in types_news.items(): 
        print(f'Crawling with {type} type')
        
        number_of_pages = count_page(f'https://batdongsan.com.vn/{part}-da-nang/p1')
        print(f'Have {number_of_pages} pages on website')
        
        exist_data += process(type, part, number_of_pages)
except Exception as e: 
    print(e)
    

# with open(f'{name_file}/{name}_{order}.json', "w", encoding="utf-8") as json_file:
#     json.dump(exist_data, json_file, ensure_ascii=False, indent=4)

Crawling with Nhà đất bán type
Have 247 pages on website
Handling on page 1
Crawling on https://batdongsan.com.vn/ban-dat-duong-quoc-lo-14b-xa-hoa-khuong/chinh-chu-ha-sap-ham-can-ban-gap-02-lien-ke-cung-mot-thua-tang-nha-go-cao-cap-09681557-pr37711601
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-nguyen-binh-khiem-phuong-an-hai-tay/ban-toa-can-ho-dich-vu-can-goc-gan-cau-rong-da-nang-pr32962850
Crawling on https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-tran-thi-ly-phuong-my-an-prj-sun-cosmo-residence/-da-nang-view-mat-tien-song-han-gio-hang-truc-tiep-cdt-chiet-khau-len-toi-19-5-pr38757226
Crawling on https://batdongsan.com.vn/ban-dat-duong-vo-van-kiet-phuong-an-hai-dong/-noi-dai-va-danh-sach-cap-nhat-xay-cao-tang-gia-canh-tranh-0932560868-pr32010256
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-le-tan-trung-phuong-tho-quang/gia-dinh-toi-chuyen-cong-tac-ban-lai-can-3-tang-dt-117-5m2-ngang-5-son-tra-gia-tot-pr38749047
Crawling on https://batdongsan.com.vn/

Crawling on https://batdongsan.com.vn/ban-dat-duong-chuong-duong-phuong-my-an/cuc-gan-1km-view-song-khu-nam-viet-a-da-nang-view-song-han-0932560868-pr34227586
Crawling on https://batdongsan.com.vn/ban-dat-duong-ngu-hanh-son-phuong-my-an/can-ban-gan-truong-dh-kinh-te-da-nang-lh-0932560868-pr30668776
Crawling on https://batdongsan.com.vn/ban-dat-duong-14b-xa-hoa-khuong/ban-khu-tai-dinh-cu-nha-nuoc-gia-1-ty-2-5m5-cach-da-nang-5-km-pr38734848
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-to-hien-thanh-phuong-an-hai-dong/cac-toa-c-ho-q-son-tra-ngu-hh-son-khu-thuong-k-nam-viet-a-son-thuy-da-ng-gia-dau-tu-pr38734730
Crawling on https://batdongsan.com.vn/ban-dat-nen-du-an-duong-tra-khe-5-phuong-hoa-hai-prj-fpt-city-da-nang/cat-lo-ban-gap-138m2-20-5m-co-loi-thoat-hiem-kdt-gia-chi-co-4-1xx-lien-he-ngay-pr38362465
Crawling on https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-pham-kiet-phuong-khue-my-prj-the-sang-residence/sieu-ban-cao-cap-2pn-tang-11-view-bien-khe-trung-tam-da-nan

Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-khue-my-dong-phuong-khue-my/can-ban-gap-toa-co-32-can-ho-co-ho-boi-khu-du-lich-sam-uat-nam-ho-xuan-huong-gia-24-ty-pr37518005
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-an-thuong-1-phuong-my-an/b-khach-s-di-bo-1-sat-bien-khe-gia-tot-pr32554545
Crawling on https://batdongsan.com.vn/ban-dat-nen-du-an-phuong-hoa-hai-prj-fpt-city-da-nang/can-chuyen-nhuong-vai-lo-phan-khu-v5-doi-lung-voi-duong-nam-ky-khoi-nghia-pr38759842
Handling on page 6
Crawling on https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-tran-hung-dao-phuong-an-hai-bac-prj-can-ho-cao-cap-azura/ban-duplex-view-song-han-va-ban-son-tra-nguoi-nuoc-ngoai-co-the-dung-ten-pr38243432
Crawling on https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-phuong-hoa-hiep-nam-prj-lakeside-palace/ban-kinh-doanh-re-t-lien-chieu-3-tang-duong-15m-ban-dat-tang-pr38370075
Crawling on https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-duong-dinh-nghe-phuong-phuoc-my

Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-phuong-thac-gian/sieu-re-mp-dien-bien-phu-ha-chao-2-ty-gia-chao-13-5-ty-co-105m2-2-ngang-5-5-tang-pr38759650
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-phuong-my-an/can-ban-khach-san-view-ban-phao-hoa-quoc-te-da-nang-pr36727782
Crawling on https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-thanh-long-phuong-thanh-binh/ban-2-mt-co-vuon-nhieu-cay-xanh-mat-tien-da-nang-gan-bien-dep-va-gia-tot-pr38603766
Crawling on https://batdongsan.com.vn/ban-dat-duong-quoc-lo-14b-xa-hoa-khuong/can-ban-da-nang-dien-tich-130m2-ma-gia-chi-6tr-m2-so-hong-co-san-bao-thue-phi-ra-so-pr38758148
Crawling on https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-me-linh-phuong-hoa-hiep-nam-prj-the-ori-garden/-2pn-view-bien-da-nang-tra-truoc-chi-666tr-lai-suat-0-pr38677869
Crawling on https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-tran-thi-ly-prj-sun-cosmo-residence/chay-hang-voi-san-pham-hang-sang-cua-group-nam-ngay-vi-tri-vang

Crawling on https://batdongsan.com.vn/ban-dat-phuong-hoa-hiep-nam/1-65-ty-co-ngay-107m2-do-thi-duong-ca-le-11-5m-khu-dan-cu-khanh-mo-rong-dan-dong-so-hong-pr37437566
Crawling on https://batdongsan.com.vn/ban-dat-duong-vu-mien-xa-hoa-phuoc/sap-2ty080-re-nhat-nam-cam-le-song-song-pham-hung-100m2-pr38757196
Crawling on https://batdongsan.com.vn/ban-dat-xa-hoa-tien/toi-chinh-chu-can-tra-no-ban-gap-lo-dia-the-dep-tai-da-nang-gia-giam-sap-san-pr38758362
Crawling on https://batdongsan.com.vn/ban-nha-mat-pho-duong-phan-chau-trinh-phuong-binh-hien/ban-3-tang-mt-doan-gan-chu-van-an-pr38757776
Crawling on https://batdongsan.com.vn/ban-nha-rieng-duong-hung-vuong-2-phuong-vinh-trung/ket-tien-ban-gap-kiet-chi-1-ty-370-trieu-pr38759148
Crawling on https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-nguyen-duy-trinh-phuong-hoa-hai/-ban-can-he-0705179-pr38744179
Crawling on https://batdongsan.com.vn/ban-loai-bat-dong-san-khac-duong-nguyen-thai-hoc-phuong-hai-chau-i/cho-han-kiot-nam-o-trung-tam-da-

...

Crawling on https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-bach-dang-phuong-phuoc-ninh/cho-3-tang-ngay-cho-han-ngang-6m-dai-18m2-trong-suot-pr38294851
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-hoang-thi-loan-phuong-hoa-minh/cho-sang-trong-voi-dien-tich-170m2-tuyen-mizuland-pr38207071
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-kinh-duong-vuong-phuong-hoa-minh/cho-nguyen-san-dien-tich-tu-100m2-tai-tuyen-mizuland-pr37859279
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-ngu-hanh-son-phuong-my-an/mb-mat-tien-12m-khu-vuc-sam-uat-hop-kd-showroom-ngan-hang-kara-q-mizuland-pr38649488
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-ngu-hanh-son-phuong-my-an/-cho-doi-dien-truong-dh-kinh-te-da-nang-san-tran-san-dieu-hoa-mizuland-pr34985121
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-giang-chau-1-phuong-khue-my/co-hoi-vang-so-huu-vp-cho-160m2-tai-khu-vuc-ngu-hanh-son-mizuland-pr37240418
Crawling on htt

Crawling on https://batdongsan.com.vn/cho-thue-sang-nhuong-cua-hang-ki-ot-duong-nguyen-thai-hoc-phuong-hai-chau-i/nhan-boong-20-trieu-gom-45-thuong-xa-cho-han-han-square-pr38756467
Crawling on https://batdongsan.com.vn/cho-thue-sang-nhuong-cua-hang-ki-ot-duong-nguyen-thai-hoc-phuong-hai-chau-i/kiot-tttm-cho-han-kinh-doanh-tai-da-nang-thu-nhap-nhu-seoul-gia-11-2-tr-thang-pr38756653
Crawling on https://batdongsan.com.vn/cho-thue-nha-rieng-duong-ong-ich-khiem-phuong-tam-thuan-prj-phu-gia-compound/chuyen-cho-compoud-da-nang-3-tang-4pn-full-nt-ngay-tai-tttp-chi-tu-20-trieu-pr38431077
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-le-van-duyet-phuong-nai-hien-dong-prj-hoa-binh-green-da-nang/chuyen-c-golden-bay-cam-ket-gia-re-nhiet-tinh-tro-24-7-goi-pr38642283
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ngo-quyen-phuong-tho-quang-prj-son-tra-ocean-view/c-2-png-ngu-huong-bien-tai-toa-cao-cap-view-pr38535853
Crawling on https://batdongsan.com.vn/ch

Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-me-linh-phuong-hoa-hiep-nam-prj-the-ori-garden/c-day-du-san-pham-1-2-3pn-va-spuse-kinh-doanh-pr37052021
Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-10-phuong-hoa-khanh-bac/-cho-500m2-1000m2-2000m2-2500m2-4500m2-5000m2-kcn-lien-chieu-da-nang-pr36655686
Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-5-phuong-hoa-tho-tay/cho-2000m-4000m-5000m-6000m-10000m-trong-khu-cong-nghiep-cam-tp-da-nang-pr36637076
Handling on page 9
Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-5-phuong-hoa-tho-tay/cho-2000m-4000m-5000m-6000m-10000m-trong-khu-cong-nghiep-cam-tp-da-nang-pr36637076
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-le-van-duyet-phuong-nai-hien-dong-prj-hoa-binh-green-da-nang/c-chungcu-sieu-dep-sieu-min-golden-bay-tan-tra-pr38294158
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-duong-loan-phuong-hoa-xuan

Crawling on https://batdongsan.com.vn/cho-thue-nha-mat-pho-pho-tran-dai-nghia-phuong-hoa-hai/cho-3-tang-moi-keng-mt-ngu-hanh-son-pr38750439
Crawling on https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-hai-phong-phuong-thach-thang/cho-capam-4-xuyen-suot-ngang-7-2m-trung-tam-sam-uat-pr38750388
Crawling on https://batdongsan.com.vn/cho-thue-sang-nhuong-cua-hang-ki-ot-duong-nguyen-thai-hoc-phuong-hai-chau-i/mo-ban-45-kiot-cho-han-han-sizang-square-chi-800-trieu-kiot-vi-tri-don-1000-luot-khach-han-ngay-pr38731172
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-vo-chi-cong-phuong-hoa-hai-prj-fpt-city-da-nang/c-2-3-png-ngu-tai-plaza-gia-5-5tr-thang-bao-phi-lh-pr38532592
Crawling on https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-dien-bien-phu-phuong-chinh-gian/cho-5-tang-trong-suot-co-thang-may-trung-tam-quan-thanh-khe-pr38025587
Crawling on https://batdongsan.com.vn/cho-thue-nha-mat-pho-duong-dung-si-thanh-khe-phuong-thanh-khe-tay/cho-lo-2mt-lon-khu-sinh-vien-do

Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-hoang-ke-viem-phuong-my-an/cho-lo-2-mt-p-dt-310m2-sat-bien-gia-chi-50tr-pr38656047
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ham-nghi-phuong-thac-gian-prj-hoang-anh-gia-lai-lake-view-residence/2pn-full-noi-that-chi-don-vao-o-kng-tham-moi-c-lan-dau-pr38593591
Handling on page 14
Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-10-phuong-hoa-tho-tay/cho-500m-860m2-1000m-1500m-2000m-2500m-3000m-10-000m2-kcn-cam-pr37667407
Crawling on https://batdongsan.com.vn/cho-thue-kho-nha-xuong-dat-duong-hoang-ke-viem-phuong-my-an/cho-lo-2-mt-p-dt-310m2-sat-bien-gia-chi-50tr-pr38656047
Crawling on https://batdongsan.com.vn/cho-thue-can-ho-chung-cu-duong-ham-nghi-phuong-thac-gian-prj-hoang-anh-gia-lai-lake-view-residence/2pn-full-noi-that-chi-don-vao-o-kng-tham-moi-c-lan-dau-pr38593591
Crawling on https://batdongsan.com.vn/cho-thue-van-phong-duong-nguyen-huu-tho-phuong-hoa-cuong-bac/c

In [6]:
exist_data

[{'post_id': '37711601',
  'Loại tin': 'Nhà đất bán',
  'Tiêu đề': 'CHủ đại hạ giá giảm gần 2 tỷ đồng, bán gấp 2 lô đất liền kề lấy tiền cho con đi du học nước ngoài',
  'Địa chỉ': 'Đường Quốc Lộ 14B, Xã Hòa Khương, Hòa Vang, Đà Nẵng.',
  'Mô tả': '- Cần bán gấp 02 lô đất liền kề cùng một thửa đất, tặng nhà gỗ cao cấp, đã có Giấy chứng nhận quyền sử dụng đất có đặc điểm như sau:\n- Hai mặt tiền gồm một mặt là quốc lộ 14B hiện nay đang được cải tạo, nâng cấp, đoạn qua địa bàn thành phố Đà Nẵng, con đường lớn nhất miền trung, là con đường thương mại nối liền các tỉnh miền trung với Tây nguyên, thửa đất cần bán thuộc điểm cuối tại khoảng Km32 + 185, nằm trên địa phận xã Hòa Khương, huyện Hòa Vang, thành phố Đà Nẵng, đây là tuyến đường trục chính đô thị, cấp đô thị, vận tốc thiết kế 80km/h với quy mô 6 làn xe, phần cầu trên tuyến được tận dụng toàn bộ, mở rộng thêm để đồng bộ với bề rộng đường; tổng số cầu mở rộng dự kiến là 4 cầu, mặt tiền còn lại là con đường 5,5m nối dài thông đến quốc 

In [7]:
with open(f'{name_file}/{name}_{order}.json', "w", encoding="utf-8") as json_file:
    json.dump(exist_data, json_file, ensure_ascii=False, indent=4)

In [8]:
# def connect_to_database(database, user, password, host, port):
#     try:
#         connection = psycopg2.connect(
#             database=database,
#             user=user,
#             password=password,
#             host=host,
#             port=port
#         )
#         print("Connected to the database")
#         return connection
#     except psycopg2.Error as e:
#         print(f"Error: Unable to connect to the database\n{e}")
#         return None

# def close_connection(connection, cursor):
#     if connection:
#         connection.close()
#         print("Connection closed")
#     if cursor:
#         cursor.close()
        
# def insert_data(data, conn, cursor, table_name, on_conflict = 'ON CONFLICT (post_id) DO NOTHING'):
#     for row in data:
#         columns = ', '.join(f'"{col}"' for col in row.keys())
#         values = ', '.join(['%s'] * len(row))
#         insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({values}) {on_conflict}'
#         cursor.execute(insert_query, tuple(row.values()))

#     conn.commit()

In [9]:
# database_name = "AI_Real_Estate"
# username = "postgres"
# password = "postgres01"
# host = "localhost"
# port = 5432

# conn = connect_to_database(database_name, username, password, host, port)
# cursor = conn.cursor()

In [10]:
# insert_data(exist_data, conn, cursor, 'batdongsan')

In [11]:
# close_connection(conn, cursor)